                                            FEATURE EGINEERING 
1) Se crea la columna de Sentiment_analysis, para categorizar en 0,1 o 2, las reseñas dadas por los usuarios en cada juego.


In [7]:
import pandas as pd
import numpy as np
import nltk

In [8]:
df_reviews = pd.read_csv('../CSV/ETL_user_review.csv', encoding='utf-8')
df_reviews.head()

,User_Id,Item_Id,Recommend,Review,Year_Posted
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,251610,True,I know what you think when you see this title ...,2014
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013


In [9]:
def tipo_de_Dato_df(dataframe):
    datos = {"Columna": [], "tipo_de_dato": [], "%_no_nulos": [], "%_nulos": [], "nulos": []}

    for columna in dataframe.columns:
        no_nulos = (dataframe[columna].count() / len(dataframe)) * 100
        datos["Columna"].append(columna)
        datos["tipo_de_dato"].append(dataframe[columna].apply(type).unique())
        datos["%_no_nulos"].append(round(no_nulos, 2))
        datos["%_nulos"].append(round(100-no_nulos, 2))
        datos["nulos"].append(dataframe[columna].isnull().sum())

    df_info = pd.DataFrame(datos)
        
    return df_info

tipo_de_Dato_df(df_reviews)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,User_Id,[<class 'str'>],100.00,0.00,0
1,Item_Id,[<class 'int'>],100.00,0.00,0
2,Recommend,[<class 'bool'>],100.00,0.00,0
3,Review,"[<class 'str'>, <class 'float'>]",99.94,0.06,27
4,Year_Posted,[<class 'int'>],100.00,0.00,0


Usaremos la libreria nltk (Natural Language Toolkit), para convertir las reseñas de cada usuario: 

In [10]:
#Se usa nltk y vader_lexicon para obtener los puntajes para cada review:

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Asignamos el analizador de sentimiento:
sia = SentimentIntensityAnalyzer()

#CSe crea una funciona para asignar valores de acuerdo a la escala
def sentiment_score(rev):
    if pd.isnull(rev) or rev == '':
        return 1  
    elif isinstance(rev, str):
        sentiment = sia.polarity_scores(rev)
        compound_score = sentiment['compound']

        if compound_score >= 0.05:
            #Umbral positivo
            return 2  
            #Umbral negativo
        elif compound_score <= -0.05:
            return 0  
        else:
             #UMbral neutro
            return 1 
    else:
        return 1

df_reviews['Review'] = df_reviews['Review'].astype(str)

#Se aplica la funcion:
df_reviews['Sentiment_Analysis'] = df_reviews['Review'].apply(sentiment_score)

df_reviews

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Asus\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,User_Id,Item_Id,Recommend,Review,Year_Posted,Sentiment_Analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2011,2
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2011,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
3,js41637,251610,True,I know what you think when you see this title ...,2014,2
4,js41637,227300,True,For a simple (it's actually not all that simpl...,2013,2
...,...,...,...,...,...,...
48493,wayfeng,730,True,its FUNNNNNNNN,2015,1
48494,76561198251004808,253980,True,Awesome fantasy game if you don't mind the gra...,2015,2
48495,72947282842,730,True,Prettyy Mad Game,2015,0
48496,ApxLGhost,730,True,AMAZING GAME 10/10,2015,2


In [11]:
tipo_de_Dato_df(df_reviews)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,User_Id,[<class 'str'>],100.0,0.0,0
1,Item_Id,[<class 'int'>],100.0,0.0,0
2,Recommend,[<class 'bool'>],100.0,0.0,0
3,Review,[<class 'str'>],100.0,0.0,0
4,Year_Posted,[<class 'int'>],100.0,0.0,0
5,Sentiment_Analysis,[<class 'int'>],100.0,0.0,0


Una vez creada nuestra columna de Sentiment analysis, dropeamos la columna Review, pues ya no será necesaria.

In [12]:
df_reviews = df_reviews.drop(columns=['Review'])
df_reviews.columns

Index(['User_Id', 'Item_Id', 'Recommend', 'Year_Posted', 'Sentiment_Analysis'], dtype='object')

In [13]:
user_reviews_sentiment = '../Parquets/Review_Sentiment_Analysis.parquet'

df_reviews.to_parquet(user_reviews_sentiment, index=False)

In [14]:
user_reviews_sentiment = '../CSV/Review_Sentiment_Analysis.csv'

df_reviews.to_csv(user_reviews_sentiment, index=False, encoding ='utf-8')